# ATEK Demo 2: ATEK Data Store and training

In [ ]:
import faulthandler

import logging
import os
from logging import StreamHandler
import numpy as np
from typing import Dict, List, Optional
import torch
import sys
import subprocess
from tqdm import tqdm

from atek.viz.atek_visualizer import NativeAtekSampleVisualizer
from atek.data_loaders.atek_wds_dataloader import (
    create_native_atek_dataloader
)
from atek.util.file_io_utils import load_yaml_and_extract_tar_list
from omegaconf import OmegaConf

faulthandler.enable()

# Configure logging to display the log messages in the notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger()

def run_command_and_display_output(command):
    # Start the process
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    
    # Poll process.stdout to show stdout live
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    rc = process.poll()
    return rc

### Set up data and code paths

In [ ]:
# data_dir = os.path.join(os.path.expanduser("~"), "Documents", "projectaria_tools_adt_data/")
data_dir = "/home/louy/Calibration_data_link/Atek/2024_08_05_DryRun"
atek_src_path = os.path.join(os.path.expanduser("~"), "atek_on_fbsource")
viz_conf = OmegaConf.load(os.path.join(atek_src_path, "atek", "configs", "obb_viz.yaml"))

## Part 1: ATEK Data Store
**ATEK Data Store** is a place where users can directly download preprocessed Aria datasets in WebDataset (WDS) format, completely skipping the preprocessing step. This demo walks through the steps for accessing datasets on ATEK Data Store. 

1. Download `AriaDigitalTwin_ATEK_download_urls.json` from [Dataverse](https://www.internalfb.com/manifold/explorer/surreal_atek_public/flat/AriaSyntheticEnvironment_ATEK_download_urls.json).  
2. Call `tools/dataverse_url_parser.py` to parse json file:
    1. [Option 1] Create streamable-version yaml files.
    2. [Option 2] Download the WDS files and create local-version yaml files. 
3. Load yaml files in ATEK lib.    

In [ ]:
# First, download json file from ATEK Data Store 
atek_json_path = os.path.join(data_dir, "AriaDigitalTwin_ATEK_download_urls.json")
if not os.path.exists(atek_json_path):
    logger.error("Please download AriaDigitalTwin_ATEK_download_urls.json from ATEK Data Store")
    exit()

# Second, parse into streamable yaml files
create_streamable_yaml_command = [
    "python3", f"{atek_src_path}/tools/dataverse_url_parser.py",
    "--config-name","cubercnn", 
    "--input-json-path",f"{atek_json_path}",
    "--output-folder-path",f"{data_dir}/streamable_yamls/",
    "--max-num-sequences", "5",
    "--train-val-split-ratio", "0.8"
]
return_code = run_command_and_display_output(create_streamable_yaml_command)

# Create ATEK data loader from streamable yaml file
tar_file_urls = load_yaml_and_extract_tar_list(yaml_path = os.path.join(data_dir, "streamable_yamls", "streamable_validation_tars.yaml"))
atek_dataloader = create_native_atek_dataloader(urls = tar_file_urls, batch_size=None, repeat_flag=False)

# Visualize ATEK WDS files in streaming mode
atek_visualizer = NativeAtekSampleVisualizer(viz_prefix = "dataloading_visualizer", conf = viz_conf)
for atek_sample_dict in atek_dataloader:
    atek_visualizer.plot_atek_sample_as_dict(atek_sample_dict)

## Part 2: ATEK Training example with CubeRCNN
User can call our `tools/train_cubercnn.py` script to do a mini-scale training on downloaded data. We will run this on my local machine for a mini demonstration.  

Core code snippets in the script (check out the script for full details): 
```
model.train()
tar_file_urls = load_yaml_and_extract_tar_list(train_list_yaml)
data_loader = create_atek_dataloader_as_cubercnn(urls = tar_file_urls, ...)\

# Loop over cubercnn-format data samples
for sample_data in data_loader:
    # Training step
    loss_dict = model(data)
    losses = sum(loss_dict.values())
    optimizer.zero_grad()
    losses.backward()
    optimizer.step()
    ...
```

In [ ]:
# Example training command
mini_training_command = [
  f"python",f"{atek_src_path}/tools/train_cubercnn.py",
  "--config-file",f"{data_dir}/cubercnn_train_config_mini_example.yaml",
  "--num-gpus", "1", 
  "OUTPUT_DIR", f"{data_dir}/mini_test_1",
  "TRAIN_LIST",f"{data_dir}/streamable_yamls/streamable_train_tars.yaml",
  "TEST_LIST", f"{data_dir}/streamable_yamls/streamable_validation_tars.yaml", 
  "CATEGORY_JSON", f"{atek_src_path}/data/atek_id_to_name.json",
  "ID_MAP_JSON", f"{atek_src_path}/data/atek_name_to_id.json", 
  "MODEL.WEIGHTS_PRETRAIN", f"/home/louy/Calibration_data_link/Atek/cubercnn_DLA34_FPN.pth"
]
return_code = run_command_and_display_output(mini_training_command)

In [ ]:
# Inspect training progress using tensorboard
tensorboard_command = ["tensorboard", f"--logdir={workdir}/mini_test_1", "--port", "6007", "--samples_per_plugin=images=1000"]
return_code = run_command_and_display_output(tensorboard_command)